In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import json

In [2]:
url =  "https://kdl.org/resource/"

html = requests.get(url)

soup = BeautifulSoup(html.content, 'html.parser')

In [3]:
block1 = soup.find_all("h3" ,class_='entry-header resource-title')

In [4]:
data_list = []

for tag in block1:
    # Grabbing the db link
    link = tag.find('a')['href']

    # Grabbing the db title
    block = tag.find('a', class_='o-link--double-chevron o-heading--medium')

    # Extract the data-analytics attribute and parse the JSON   
    data_analytics = json.loads(block['data-analytics'])
    
    # Extract the web_content_title from json
    title = data_analytics[0]['entities']['web_content']['web_content_title']

    # Append to the list as a dictionary
    data_list.append({'title': title, 'link': link})

# Create DataFrame from the list of dictionaries
the_df = pd.DataFrame(data_list)

In [5]:
desc_list = []

block2 = soup.find_all("div" ,class_='or-description') # This resource is only available from within a library building. PROBLEM

for desc in block2:
    #print(desc, "\n")
    description = desc.find("p").get_text(strip=True)
    
    desc_list.append(description)

In [6]:
# combine all info together 

the_df = the_df.assign(Description = desc_list)

In [7]:
# saving options for the KDL database db

# convert df to JSON
json_data = the_df.to_json(orient='records')

# save the JSON data to a file
with open('databaesDB.json', 'w') as f:
    f.write(json_data)

#file_path = r"data\databasedb.csv"

#the_df.to_excel(file_path, index=False)
#the_df.to_csv(file_path, index=False)